In [1]:
%reset -f
import os
from dotenv import load_dotenv
from datetime import datetime

# Get the SEO keywords and parent folder as input from the user
seo_keywords = "10 Elegant Christmas Nails" 
parent_folder = "elegant-christmas-nails"
# 1: moroccoheritage 2 : gobitcode
website = "1"
current_date = datetime.now().strftime('%Y-%m-%d')
# app requirements
blog_path = "/home/adil/repo/morocco-heritage/data/blog/"
image_path = "/home/adil/repo/morocco-heritage/public/static/images"

In [ ]:
if website == '1':
    file_path = f'./format/moroccoheritage.mdx'
else:
    file_path = f'./format/gobitcode.mdx'

with open(file_path, 'r') as file:
    mdx_format = file.read()


mdx_format = mdx_format.replace("current_date", current_date)

# Load environment variables from .env file
load_dotenv()

# Get the API key from the environment variable
groq_api_key = os.getenv('GROQ_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')
google_cx = os.getenv('GOOGLE_CX')


In [ ]:
from groq import Groq




client = Groq(api_key=groq_api_key)


completion = client.chat.completions.create(
    model="llama3-70b-8192",
    messages=[
        {
            "role": "system",
            "content": f"You are a blog writer who write blog in this format: \n {mdx_format} \n Do not include any introductory text "
        },
        {
            "role": "user",
            "content": f"Write a long-form blog about '{seo_keywords}' \n Do not forget image links and alt text"
        }
    ],
    temperature=1,
    max_tokens=1024*4,
    top_p=1,
    stream=None,
    stop=None,
)
# opencv cnn face recognition

mdx_blog = ""
for choice in completion.choices:
    mdx_blog += choice.message.content

print(mdx_blog)

In [ ]:
import re
# Function to generate a slug from the title
def generate_slug(title):
    # Convert to lowercase
    slug = title.lower()
    # Replace spaces and special characters with hyphens
    slug = re.sub(r'[^a-z0-9]+', '-', slug)
    # Remove leading and trailing hyphens
    slug = slug.strip('-')
    return slug


# Regular expression to extract the title
title_match = re.search(r'^title:\s*(.*)$', mdx_blog, re.MULTILINE)

if title_match:
    title = title_match.group(1)
    slug = generate_slug(title)
else:
    print("Title not found")

print(f"Slug: {slug}")

In [ ]:
from google_images_search import GoogleImagesSearch

def download_images(keyword,path,  limit=1):
    # You need to get your own API key and CX from Google Custom Search JSON API
    gis = GoogleImagesSearch(google_api_key,google_cx)

    search_params = {
        'q': keyword,
        'num': limit,
        'safe': 'off',
        'fileType': 'jpg|png',
        'imgType': 'photo',
        'imgSize': 'LARGE',  # Use a single size or remove this line if not needed
        # 'imgDominantColor': 'black'  # Use a single color or remove this line if not needed
    }

    gis.search(search_params=search_params)
    image = gis.results()[0]
    image_save_path = os.path.join(image_path,path)
    image.download(image_save_path)
    image_name = os.path.basename(image.path)
    return image_name




In [ ]:



# Regex to match the <img> tag and capture the alt attribute
img_tag_regex = r'<img[^>]*alt="([^"]*)"[^>]*>'

# Find all matches
matches = re.findall(img_tag_regex, mdx_blog)



# Print the alt attributes
for match in matches:
    image_name =  download_images(match , parent_folder)
    print(f"alt attribute: {match}")
    new_src = f'/static/images/{parent_folder}/{image_name}'
    mdx_blog = re.sub(rf'(<img[^>]*src=")[^"]*("[^>]*alt="{re.escape(match)}"[^>]*>)', rf'\1{new_src}\2', mdx_blog)
    
print(mdx_blog) 
    

In [ ]:
#correct : in title
# Sample string


# Regex to match and remove the colon from the title line
title_regex = r'^(title:.*?):(.*)$'

# Remove the colon from the title line
mdx_blog = re.sub(title_regex, r'\1\2', mdx_blog, flags=re.MULTILINE)

print(mdx_blog)

In [ ]:
filename = f"{slug}.mdx"

blog_save_path = os.path.join(blog_path, filename)

with open(blog_save_path, 'w') as file:
    file.write(mdx_blog)

print(f"File '{filename}' created successfully.")